In [1]:
import pandas as pd
import json
import pyodbc

In [2]:
region_a_sales = r"Data\order_region_a.xlsx"
df_region_a_sales = pd.read_excel(region_a_sales)

In [3]:
region_b_sales = r"Data\order_region_b.xlsx"
df_region_b_sales = pd.read_excel(region_b_sales)

In [4]:
df_region_a_sales["region"]="regin_a"
df_region_b_sales["region"]="regin_b"
df_both_regions = pd.concat([df_region_a_sales, df_region_b_sales], axis=0, ignore_index=True)

In [5]:
df_both_regions["total_sales"] = df_both_regions["QuantityOrdered"]*df_both_regions["ItemPrice"]

In [6]:
df_both_regions["net_sales"]=df_both_regions["total_sales"]-df_both_regions['PromotionDiscount'].apply(lambda x: float(json.loads(x)['Amount']))

In [7]:
df_both_regions = df_both_regions[df_both_regions["total_sales"] > 0]

In [8]:
df_both_regions = df_both_regions.drop_duplicates()
df_both_regions = df_both_regions.sort_values(by='net_sales', ascending=False)

In [9]:
df_both_regions.sort_values(by="batch_id")

,OrderId,OrderItemId,QuantityOrdered,ItemPrice,PromotionDiscount,batch_id,region,total_sales,net_sales
1844,171-1219791-5921968,20195300547395,1,349.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",1,regin_a,349.0,339.0
59413,171-9879807-9579511,51783867130851,1,629.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",1,regin_b,629.0,618.9
67235,402-4937433-9630760,21331241716755,1,1199.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",1,regin_b,1199.0,1188.9
19751,402-2943894-4329924,40784141968123,1,999.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",1,regin_a,999.0,988.9
45708,171-0753667-1825903,23546657077203,1,529.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",1,regin_b,529.0,519.0
...,...,...,...,...,...,...,...,...,...
3999,171-2675921-4044368,55220922012515,1,1359.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",2447,regin_a,1359.0,1348.9
56695,171-8088488-0001913,6193743072587,1,689.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",2449,regin_b,689.0,678.9
12201,171-8088488-0001913,6193743072587,1,689.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",2449,regin_a,689.0,678.9
24366,402-5961962-5542738,12629170834723,1,649.0,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",2450,regin_a,649.0,639.0


In [10]:
DRIVER = "ODBC Driver 17 for SQL Server"
SERVER = "QRA-SIVANAGABAB"
DATABASE = "MyDataBase"
USERNAME = "naga"
PASSWORD = "1234"

try:
    # Connect to SQL Server
    connection = pyodbc.connect(
        f"DRIVER={{{DRIVER}}};"
        f"SERVER={SERVER};"
        f"DATABASE={DATABASE};"
        f"UID={USERNAME};"
        f"PWD={PASSWORD}"
    )
    print("Connection success!")
except Exception as e:
    print("Connection failed:", e)


Connection success!


In [11]:
cursor = connection.cursor()

In [12]:
insert_query = """
INSERT INTO Orders (OrderId, OrderItemId, QuantityOrdered, ItemPrice, PromotionDiscount, batch_id, region, total_sales, net_sales)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

In [13]:
for index, row in df_both_regions.iterrows():
    cursor.execute(insert_query, 
                   row['OrderId'], 
                   row['OrderItemId'], 
                   row['QuantityOrdered'], 
                   row['ItemPrice'], 
                   row['PromotionDiscount'], 
                   row['batch_id'], 
                   row['region'], 
                   row['total_sales'], 
                   row['net_sales'])

connection.commit()

In [14]:
select_query = """
SELECT [OrderIndexID]
    ,[OrderId]
    ,[OrderItemId]
    ,[QuantityOrdered]
    ,[ItemPrice]
    ,[PromotionDiscount]
    ,[batch_id]
    ,[region]
    ,[total_sales]
    ,[net_sales]
FROM [dbo].[Orders]
"""

cursor.execute(select_query)

rows = cursor.fetchall()
columns = [column[0] for column in cursor.description]

df_from_db = pd.DataFrame.from_records(rows, columns=columns)

df_from_db

,OrderIndexID,OrderId,OrderItemId,QuantityOrdered,ItemPrice,PromotionDiscount,batch_id,region,total_sales,net_sales
0,1000,403-4293760-0003568,3028804725147,30,12686.40,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",1295,regin_b,380592.00,380581.90
1,1001,403-4293760-0003568,3028804725147,30,12686.40,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",1295,regin_a,380592.00,380581.90
2,1002,402-9401037-0972351,36243216037483,20,9980.00,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",614,regin_b,199600.00,199590.00
3,1003,402-9401037-0972351,36243216037483,20,9980.00,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",614,regin_a,199600.00,199590.00
4,1004,403-9316417-2694764,38020718542787,20,9980.00,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",811,regin_a,199600.00,199589.90
...,...,...,...,...,...,...,...,...,...,...
88873,89873,171-2145921-6835550,58816910117843,1,159.00,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10""}",268,regin_a,159.00,149.00
88874,89874,171-0190027-9576360,58032842563083,1,159.00,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",37,regin_a,159.00,148.90
88875,89875,171-0190027-9576360,58032842563083,1,159.00,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",37,regin_b,159.00,148.90
88876,89876,403-2807164-7584363,41628839338355,1,159.00,"{ ""CurrencyCode"": ""INR"", ""Amount"": ""10.1""}",150,regin_b,159.00,148.90


In [15]:
cursor.close()
connection.close()